## Imports

In [99]:
# General imports
import time
import json
import random
import os

# Intent recognition imports
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# ChatGPT imports
%load_ext autoreload
%autoreload 2
from gpt import GPT
from settings import Settings
from intent_model import IntentModel

# Chat demo
import gradio as gr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data 

In [100]:
# Load the data from JSON file
with open("intent_recognition.json", "r") as f:
    data = json.load(f)

train_data = []
for datum in data:
    texts = datum["train_questions"]
    label = datum["intent"]
    for text in texts:
        train_data.append((text, label))

# Define the mapping between top-level labels and integers
label_map = {label: i for i, label in enumerate(set([data[1] for data in train_data]))}

# Convert the training data labels to integers using the label_map
labels = torch.tensor([label_map[data[1]] for data in train_data])
    
print(train_data)
print(label_map)
print(labels)

[('Could you provide a summary of your privacy policy?', 'summarization_privacy'), ('What are the key points of your privacy policy?', 'summarization_privacy'), ('In a few sentences, what does your privacy policy say?', 'summarization_privacy'), ('Can you give me an overview of your privacy policy?', 'summarization_privacy'), ('What is your privacy policy about?', 'summarization_privacy'), ('Can you give me a brief explanation of your privacy policy?', 'summarization_privacy'), ('How do I opt out of data sharing?', 'opt_out'), ('Is there a way to opt out of sharing my data?', 'opt_out'), ('What are my options for opting out of data sharing?', 'opt_out'), ('Can you explain how to opt out of data sharing?', 'opt_out'), ('How can I prevent my data from being shared?', 'opt_out'), ('Can I choose not to have my data shared?', 'opt_out'), ('How can I turn off the notifications?', 'change_settings'), ('Is there a way to enable the wifi?', 'change_settings'), ('What steps do I need to take to 

# Intent recognition model

In [101]:
intent_model = IntentModel(train_data, label_map)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1, Loss: 1.039520025253296
Epoch 2, Loss: 1.4394302368164062
Epoch 3, Loss: 1.076171875
Epoch 4, Loss: 1.168786883354187
Epoch 5, Loss: 1.087881088256836
Epoch 6, Loss: 1.461206078529358
Epoch 7, Loss: 1.0152387619018555
Epoch 8, Loss: 1.0576425790786743
Epoch 9, Loss: 1.061463713645935
Epoch 10, Loss: 1.1557025909423828
Epoch 11, Loss: 1.2461787462234497
Epoch 12, Loss: 1.1195636987686157
Epoch 13, Loss: 1.0729016065597534
Epoch 14, Loss: 1.147338628768921
Epoch 15, Loss: 1.060958743095398
Epoch 16, Loss: 1.0472875833511353
Epoch 17, Loss: 0.8990664482116699
Epoch 18, Loss: 1.0757272243499756
Epoch 19, Loss: 1.0523760318756104
Epoch 20, Loss: 1.1434725522994995


In [102]:
print(intent_model.get_intent("How can I turn off my notifications?"))
print(intent_model.get_intent("How can I [state] the [setting]?"))


change_settings
change_settings


## ChatGPT model

In [103]:
# GPT model here
# gpt_model = GPT()

gpt_model = False

# Test ChatGPT model
# gpt_model.answer_question(question='What is the most important thing I need to know about your privacy statement?')


# Load Settings class

In [104]:
# settings_model = Settings()

# user_id = input("Enter the ID of the existing user from the JSON file: ")
# settings_model.set_current_user(user_id)
# security_level = settings_model.estimate_user_security_level()
# print("User security level:", security_level)

## Get response using intent

In [105]:
def get_object_by_intent(intent):
    for object in data:
        if object['intent'] == intent:
            return object
    return None

# Set up gradio chat for demo

In [106]:
theme = gr.themes.Soft(
    primary_hue="orange",
    secondary_hue="orange",
).set(
    body_text_color_dark='*neutral_800',
    background_fill_primary_dark='*neutral_50',
    background_fill_secondary_dark='*neutral_50',
    border_color_accent_dark='*primary_300',
    border_color_primary_dark='*neutral_200',
    color_accent_soft_dark='*primary_50',
    link_text_color_dark='*secondary_600',
    link_text_color_active_dark='*secondary_600',
    link_text_color_hover_dark='*secondary_700',
    link_text_color_visited_dark='*secondary_500',
    block_background_fill='*neutral_100',
    block_background_fill_dark='*neutral_100',
    block_label_background_fill='*primary_400',
    block_label_background_fill_dark='*primary_400',
    block_label_text_color='*neutral_50',
    block_label_text_color_dark='*neutral_50',
    block_title_text_color='*neutral_50',
    block_title_text_color_dark='*neutral_50',
    checkbox_background_color_dark='*background_fill_primary',
    checkbox_background_color_selected='*primary_500',
    checkbox_background_color_selected_dark='*primary_500',
    checkbox_border_color_dark='*neutral_100',
    checkbox_border_color_focus='*primary_300',
    checkbox_border_color_focus_dark='*primary_300',
    checkbox_border_color_hover_dark='*neutral_300',
    checkbox_border_color_selected='*primary_500',
    checkbox_border_color_selected_dark='*primary_500',
    checkbox_border_width_dark='1px',
    checkbox_label_background_fill_selected_dark='*primary_500',
    checkbox_label_text_color_selected_dark='white',
    error_background_fill_dark='#fee2e2',
    error_border_color_dark='#fecaca',
    input_background_fill_dark='white',
    input_background_fill_focus_dark='*secondary_500',
    input_border_color_dark='*neutral_50',
    input_border_color_focus_dark='*secondary_300',
    input_placeholder_color_dark='*neutral_400',
    slider_color_dark='*primary_500',
    stat_background_fill_dark='*primary_300',
    table_border_color_dark='*neutral_300',
    table_even_background_fill_dark='white',
    table_odd_background_fill_dark='*neutral_50',
    button_primary_background_fill_dark='*primary_500',
    button_primary_background_fill_hover_dark='*primary_400',
    button_primary_border_color_dark='*primary_200',
    button_secondary_background_fill_dark='white',
    button_secondary_background_fill_hover_dark='*neutral_100',
    button_secondary_border_color_dark='*neutral_200',
    button_secondary_text_color_dark='*neutral_800'
)

with gr.Blocks(theme=theme, css="chat/chat.css") as demo:
    gr.Image("https://iyyu.com/_nuxt/img/navbar_logoW@2x.79eba99.png", interactive=False, tool="image", show_label=False, elem_classes="logo").style(width=200)
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    submit = gr.Button("Submit")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]

        intent = get_intent(question=user_message)
        print('intent:', intent)
        
        intent =  get_object_by_intent(intent)
        
        # generating a response with GPT if the main intent was 'privacy_policy' or 'legal_statement'
        use_gpt = intent['use_gpt']

        # if the intent is 'privacy_policy' or 'legal_statement', use GPT to generate a response
        response = '(ChatGPT - intent found: {}): \n{}'.format(intent['intent'], "ChatGPT response here") if use_gpt else '(intent found: {}): \n{}'.format(intent['intent'], intent['responses'][0])
        # response = '(ChatGPT - intent found: {}): \n{}'.format(intent['intent'], gpt_model.answer_question(question=user_message)) if use_gpt else '(intent found: {}): \n{}'.format(intent['intent'], intent['responses'][0])


        # response = random.choice(response_map[intent])
        history[-1][1] = response
        # The sleep is to simulate a more natural conversation
        time.sleep(1)
        return history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    submit.click(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    
demo.launch()

Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.
